[LSTMによる時系列データの予測](http://sweng.web.fc2.com/ja/program/python/time-series-forecast-lstm.html)

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import pandas as pd
from matplotlib import pylab as plt
import seaborn as sns
%matplotlib inline
sns.set()

Using TensorFlow backend.


### 簡単な例
まずは、簡単な例で試します。 3つの入力から、2つの値を出力(予測)することにします。

LSTM (Long short-term memory) を用いて、時系列データの予測を行います 
いくつかモジュールが必要なので、まとめてimportしておきます

In [2]:
# 学習用データ。xが入力、yが出力(答え)です。
 
x = np.array([[10,20,30], [20,30,40], [30, 40, 50], [40, 50, 60]])
y = np.array([[40, 50], [50, 60], [60, 70], [70, 80]])
 
# 行列のフォーマット変更。
# LSTMは、入力フォーマットを[サンプルの数, 入力のステップ数(この場合は3), features]とする必要があるためです。
x = x.reshape((x.shape[0], x.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

### 次にネットワークを構築します。

In [3]:
# Sequentialは、あるレイヤの全ノードと、次レイヤの全ノードをつなぐDNNのモデルです。
m = Sequential()
# DNNの第一レイヤとして、LSTMを追加します。第一引数は出力の次元数です。
# ここでは100としています。activationは活性化関数で、ここではReLUを使うように設定しています。
# input_shapeは、入力データのフォーマットです。
m.add(LSTM(100, activation='relu', input_shape=(3, 1)))
# RepeatVectorにより、入力を繰り返します。ここでの繰り返し回数は、予測範囲(今回は2データ)となります。
m.add(RepeatVector(2))
# 再びLSTM。ただし、ここではreturn_sequences=Trueを指定します。
m.add(LSTM(100, activation='relu', return_sequences=True))
# TimeDistributedを指定し、かつ、Dense(1)で、出力の次元数を「１」に指定します。
m.add(TimeDistributed(Dense(1)))
# 最後にcompileメソッドで、学習時の最適化手法や、損失関数を指定します。
# ここでは最適化手法としてAdamを、損失関数としてMSE(Mean Squared Error; 平均二乗誤差)を指定します。
m.compile(optimizer='adam', loss='mse')

### fitメソッドで学習を行います。

In [4]:
# 学習。時間が少しかかる可能性があります。
m.fit(x, y, epochs=1000, verbose=0)

InternalError:  Blas GEMM launch failed : a.shape=(4, 100), b.shape=(100, 400), m=4, n=400, k=100
	 [[{{node lstm_1/while/body/_1/MatMul_1}}]] [Op:__inference_keras_scratch_graph_2613]

Function call stack:
keras_scratch_graph


学習済みのモデルに、[50, 60, 70]という入力を与えて、結果がどうなるかを見てみます。 理想では[80, 90]となればOKです。

In [10]:
x_input = np.array([50, 60, 70])
x_input = x_input.reshape((1, 3, 1))
yhat = m.predict(x_input)
print(yhat)

[[[81.256874]
  [90.78984 ]]]


結果は以下です。
[[[82.211136]
  [93.43616 ]]]